<a href="https://colab.research.google.com/github/1vanl0pez/DataScienceSantiagoIvan/blob/main/DSPRO1_HRAnalytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Import libraries for data management and math operations
import pandas as pd
import numpy as np
# Import the random forest tool for regression models
from sklearn.ensemble import RandomForestRegressor
# Import the tool for K fold crros model validation
from sklearn.model_selection import KFold
# Import the tool to compute MSE
from sklearn.metrics import mean_squared_error

In [4]:
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/general_data.xlsx'

Mounted at /content/drive


In [7]:
# Read the dataframe
df = pd.read_excel(file_path)
# Display the headers and the first values to confirm a correct read
df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,JobLevel,JobRole,...,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating,Unnamed: 25,Unnamed: 26
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,Female,1,Healthcare Representative,...,1,0,0,3.0,4.0,2.0,3,3,NaN,NaN
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,Female,1,Research Scientist,...,5,1,4,3.0,2.0,4.0,2,4,NaN,111.0
2,32,No,Travel_Frequently,Research & Development,17,4,Other,Male,4,Sales Executive,...,5,0,3,2.0,2.0,1.0,3,3,NaN,NaN
3,38,No,Non-Travel,Research & Development,2,5,Life Sciences,Male,3,Human Resources,...,8,7,5,4.0,4.0,3.0,2,3,NaN,NaN
4,32,No,Travel_Rarely,Research & Development,10,1,Medical,Male,1,Sales Executive,...,6,0,4,4.0,1.0,3.0,3,3,NaN,NaN


In [ ]:
# Compute main statistics
df.describe()

,Age,DistanceFromHome,Education,JobLevel,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
count,4410.000000,4410.000000,4410.000000,4410.000000,4410.000000,4391.000000,4410.000000,4410.000000,4401.000000,4410.000000,4410.000000,4410.000000,4410.000000,4385.000000,4390.000000,4372.000000,4410.000000,4410.000000
mean,36.923810,9.192517,2.912925,2.063946,65029.312925,2.694830,15.209524,0.793878,11.279936,2.799320,7.008163,2.187755,4.123129,2.723603,2.728246,2.761436,2.729932,3.153741
std,9.133301,8.105026,1.023933,1.106689,47068.888559,2.498887,3.659108,0.851883,7.782222,1.288978,6.125135,3.221699,3.567327,1.092756,1.101253,0.706245,0.711400,0.360742
min,18.000000,1.000000,1.000000,1.000000,10090.000000,0.000000,11.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,3.000000
25%,30.000000,2.000000,2.000000,1.000000,29110.000000,1.000000,12.000000,0.000000,6.000000,2.000000,3.000000,0.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000
50%,36.000000,7.000000,3.000000,2.000000,49190.000000,2.000000,14.000000,1.000000,10.000000,3.000000,5.000000,1.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
75%,43.000000,14.000000,4.000000,3.000000,83800.000000,4.000000,18.000000,1.000000,15.000000,3.000000,9.000000,3.000000,7.000000,4.000000,4.000000,3.000000,3.000000,3.000000
max,60.000000,29.000000,5.000000,5.000000,199990.000000,9.000000,25.000000,3.000000,40.000000,6.000000,40.000000,15.000000,17.000000,4.000000,4.000000,4.000000,4.000000,4.000000


In [ ]:
# Display the data type for each variable
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4410 entries, 0 to 4409
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      4410 non-null   int64  
 1   Attrition                4410 non-null   object 
 2   BusinessTravel           4410 non-null   object 
 3   Department               4410 non-null   object 
 4   DistanceFromHome         4410 non-null   int64  
 5   Education                4410 non-null   int64  
 6   EducationField           4410 non-null   object 
 7   Gender                   4410 non-null   object 
 8   JobLevel                 4410 non-null   int64  
 9   JobRole                  4410 non-null   object 
 10  MaritalStatus            4410 non-null   object 
 11  MonthlyIncome            4410 non-null   int64  
 12  NumCompaniesWorked       4391 non-null   float64
 13  PercentSalaryHike        4410 non-null   int64  
 14  StockOptionLevel        

In [ ]:
# DATA CLEANING
# Step 1: Handle missing values
# Replace missing numeric values with the median of the respective column.
# Median is more resilient to outliers than the mean.

# Look if the null values are greater than 0, if so, add them into a list
columns_with_na = df.columns[df.isnull().sum() > 0].tolist()

# Replacement
for col in columns_with_na:
    # Calculate median of the column
    median_value = df[col].median()
    # if there is a blank space, fill it with the median
    df.fillna({col: median_value}, inplace=True)

In [ ]:
# Step 2: Convert categorical variables to numerical format using one-hot encoding
# drop_first=True avoids logical errors (multicollinearity)
df = pd.get_dummies(df, drop_first=True)

# Check the cleaned dataset structure
df.head(), df.isnull().sum().sum()  # Checking for any remaining missing values

(   Age  DistanceFromHome  Education  JobLevel  MonthlyIncome  \
 0   51                 6          2         1         131160   
 1   31                10          1         1          41890   
 2   32                17          4         4         193280   
 3   38                 2          5         3          83210   
 4   32                10          1         1          23420   
 
    NumCompaniesWorked  PercentSalaryHike  StockOptionLevel  TotalWorkingYears  \
 0                 1.0                 11                 0                1.0   
 1                 0.0                 23                 1                6.0   
 2                 1.0                 15                 3                5.0   
 3                 3.0                 11                 3               13.0   
 4                 4.0                 12                 2                9.0   
 
    TrainingTimesLastYear  ...  JobRole_Human Resources  \
 0                      6  ...                    False

In [ ]:
# MODEL CONSTRUCTION
# Separate the data into two groups: employees who left (Attrition = Yes) and those who stayed (Attrition = No)
data_left = df[df['Attrition_Yes'] == 1]
data_stayed = df[df['Attrition_Yes'] == 0]

# Define the output and input variables for each data set
# For employees who left
# Remove JobSatisfaction from the input variables since it is the output variable
X_left = data_left.drop(columns=["JobSatisfaction"])
# The output variable is JobSatisfaction
y_left = data_left["JobSatisfaction"]

# For employees who stayed
# Remove JobSatisfaction from the input variables since it is the output variable
X_stayed = data_stayed.drop(columns=["JobSatisfaction"])
# The output variable is JobSatisfaction
y_stayed = data_stayed["JobSatisfaction"]

In [ ]:
# Initialize k fold cross-validation with 5 folds
# shuffle=True mixes the date before making the fold to guarantee a balanced sample
# random_state=24 is the seed, it determines reproducibility and should keep an integer constant value
kf = KFold(n_splits=5, shuffle=True, random_state=24)

# Function to perform k fold cross-validation and register feature importance
def calculate_feature_importances(X, y):
    # Initialize lists to store importances and errors
    fold_importances = []
    fold_errors = []
    # Loop through each fold
    for train_index, test_index in kf.split(X):
        # Split data into train and test sets for the current fold
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        # Initialize a random forest of 100 trees and a constant value for the seed
        model = RandomForestRegressor(n_estimators=100, random_state=24)
        # Train the model with the training set
        model.fit(X_train, y_train)
        # Predict using the test set of X values
        y_pred = model.predict(X_test)
        # Compute MSE between the predicted value and the real one and add it to a list
        fold_errors.append(mean_squared_error(y_test, y_pred))
        # Compute Gini impurity for each feature and add it to a list
        fold_importances.append(model.feature_importances_)
    # Average feature importances and errors across all folds
    avg_importances = np.mean(fold_importances, axis=0)
    avg_error = np.mean(fold_errors)
    # Return the average importances and error
    return avg_importances, avg_error

# Calculate feature importances for each group
importances_left, error_left = calculate_feature_importances(X_left, y_left)
importances_stayed, error_stayed = calculate_feature_importances(X_stayed, y_stayed)
# Store the feature importances in DataFrames for better visualization
feature_importances_left = pd.DataFrame({"Feature": X_left.columns, "Importance": importances_left}).sort_values(by="Importance", ascending=False)
feature_importances_stayed = pd.DataFrame({"Feature": X_stayed.columns, "Importance": importances_stayed}).sort_values(by="Importance", ascending=False)
# Display results
print("Top 10 most relevant features for job satisfaction")
print("\nFor employees who left")
print(feature_importances_left.head(10))
print("\nMean Squared Error for employees who left:", error_left)
print("\nFor employees who stayed:")
print(feature_importances_stayed.head(10))
print("\nMean Squared Error for emplyees who stayed:", error_stayed)

Top 10 most relevant features for job satisfaction

For employees who left
                    Feature  Importance
4             MonthlyIncome    0.117753
6         PercentSalaryHike    0.092387
1          DistanceFromHome    0.067898
8         TotalWorkingYears    0.064408
0                       Age    0.062586
5        NumCompaniesWorked    0.048119
10           YearsAtCompany    0.047797
12     YearsWithCurrManager    0.043407
11  YearsSinceLastPromotion    0.041773
2                 Education    0.039101

Mean Squared Error for employees who left: 0.1508530915176708

For employees who stayed:
                    Feature  Importance
4             MonthlyIncome    0.120501
0                       Age    0.079813
1          DistanceFromHome    0.078714
8         TotalWorkingYears    0.064464
6         PercentSalaryHike    0.061626
10           YearsAtCompany    0.052133
12     YearsWithCurrManager    0.047330
5        NumCompaniesWorked    0.044994
11  YearsSinceLastPromotion    0.04

In [ ]:
# Select from previous step the most logical relevant features into a list
top_5_features = X_stayed[["MonthlyIncome", "DistanceFromHome", "PercentSalaryHike",
                           "YearsSinceLastPromotion", "TrainingTimesLastYear"]]
# Display the features along with their range and mean for bettter comprehension
print("Chosen relevant features for job satisfaction:")
for feature in top_5_features:
    min_val = data_stayed[feature].min()
    max_val = data_stayed[feature].max()
    mean_val = data_stayed[feature].mean()
    print(f"{feature} - Min: {min_val}, Max: {max_val}, Promedio: {mean_val:.2f}")
# Declare the predictor variables
X_important_stayed = data_stayed[top_5_features.columns]
# Train the model with the selected features, 100 trees and the same seed
final_model = RandomForestRegressor(n_estimators=100, random_state=24)
final_model.fit(X_important_stayed, y_stayed)
# Function to ask for input values and predict satisfaction
def predict_satisfaction():
    print("Job satisfaction prediction")
    print("Enter the following data to determine job satisfaction:")
    # Initialize a list for input data
    input_data = []
    # Store the recieved data in the list
    for feature in top_5_features:
        value = float(input(f"{feature}: "))
        input_data.append(value)
    # Convert to numpy array for data processing
    input_data = np.array(input_data).reshape(1, -1)
    # Try the model with the input data
    satisfaction_prediction = final_model.predict(input_data)[0]
    # Classify the result into a traffic light scale
    if satisfaction_prediction <= 2:
        traffic_light = "Red"
    elif 2 < satisfaction_prediction <= 3:
        traffic_light = "Yellow"
    elif 3 < satisfaction_prediction <= 4:
        traffic_light = "Green"
    else:
        traffic_light = "Blue"
    # Display the results
    print(f"Predicted satisfaction: {satisfaction_prediction:.2f}")
    print(f"Traffic light scale: {traffic_light}")
# Call the function to predict
predict_satisfaction()

Chosen relevant features for job satisfaction:
MonthlyIncome - Min: 10510, Max: 199990, Promedio: 65672.60
DistanceFromHome - Min: 1, Max: 29, Promedio: 9.23
PercentSalaryHike - Min: 11, Max: 25, Promedio: 15.16
YearsSinceLastPromotion - Min: 0, Max: 15, Promedio: 2.23
TrainingTimesLastYear - Min: 0, Max: 6, Promedio: 2.83
Job satisfaction prediction
Enter the following data to determine job satisfaction:
MonthlyIncome: 200000
DistanceFromHome: 2
PercentSalaryHike: 29
YearsSinceLastPromotion: 0
TrainingTimesLastYear: 8
Predicted satisfaction: 2.06
Traffic light scale: Yellow


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
